# Работа с файлами

In [1]:
file = open('data/text_w.txt','w')
file.write('Hello \n')
file.write('World')

file.close()

In [2]:
file = open('data/text_w.txt','r')

print(type(file.read()))
print(file.read())

file.close()

<class 'str'>



In [3]:
file = open('data/text_w.txt','r')

print(file.read(4))
print(file.read())

file.close()

Hell
o 
World


In [4]:
file = open('data/text_w.txt','r')

for line in file:
    print(line, end = '')

file.close()

Hello 
World

In [5]:
file = open('data/text_a.txt','a')

file.write('Hello World\n')

file.close()

При работе с файлами удобно пользоваться менеджером «With ... as».
При вызове этой конструкции, внутри блока выполняются действия внутри файла, при этом после выполнения блока файл автоматически закрывается, поэтому строку ```file.close()``` прописывать не нужно

In [6]:
data = input('Введите текст: ')

with open('data/text_input.txt','a') as file:
    file.write(data+'\n')

В библиотеке pandas имеется ряд функций для чтения табличных данных,
представленных в виде объекта DataFrame. Импортируем ее.

In [7]:
import pandas as pd

Прочтем файл ex1.csv в папке data с помощью функции read_csv:

In [8]:
pd.read_csv('data/ex1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Можно было бы также воспользоваться функцией read_table, указав раз-
делитель:

In [9]:
pd.read_table('data/ex1.csv', sep=';')

,"a,b,c,d,message"
0,"1,2,3,4,hello"
1,"5,6,7,8,world"
2,"9,10,11,12,foo"


В файле не всегда есть строка-заголовок. Рассмотрим файл ex2.csv, содержащий:

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Прочитать его можно двумя способами. Можно поручить pandas выбрать
имена столбцов по умолчанию, а можно задать их самостоятельно:

In [10]:
pd.read_csv('data/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
pd.read_csv('data/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Допустим, мы хотим, чтобы столбец message стал индексом возвращаемого объекта DataFrame. Этого можно добиться, задав аргумент index_col, в котором указать, что индексом будет столбец с номером 4 или с именем 'message':

In [12]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Если вы хотите сформировать иерархический индекс из нескольких столбцов, то просто передайте список их номеров или имен. Так, например, для файла data/ex3.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16

In [13]:
pd.read_csv('data/ex3.csv', index_col=['key1', 'key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Иногда в таблице нет фиксированного разделителя, а для разделения полей используются пробелы или еще какой-то символ. В таком случае можно передать функции read_table регулярное выражение вместо разделителя. Рассмотрим текстовый файл data/ex3.txt:

         A         B         C
aaa –0.264438 –1.026059 –0.619500
bbb 0.927272 0.302904 –0.032399
ccc –0.264273 –0.386314 –0.217601
ddd –0.871858 –0.348382 1.100491

В данном случае поля разделены переменным числом пробелов и, хотя можно было бы переформатировать данные вручную, проще передать функции read_table регулярное выражение \s+ в качестве разделителя:

In [14]:
pd.read_table('data/ex3.txt', sep='\s+')

,A,B,C
aaa,–0.264438,–1.026059,–0.619500
bbb,0.927272,0.302904,–0.032399
ccc,–0.264273,–0.386314,–0.217601
ddd,–0.871858,–0.348382,1.100491


Поскольку имен столбцов на одно меньше, чем строк, read_table делает вывод, что в данном частном случае первый столбец должен быть индексом DataFrame.

У функций разбора много дополнительных аргументов, которые помогают справиться с широким разнообразием файловых форматов (см. табл.). Так, например, в файле data/ex4.csv, который имеет следующий формат:

# привет!
a,b,c,d,message
# хотелось немного усложнить тебе жизнь,
# а нечего читать CSV–файлы на компьютере
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Параметр skiprows позволяет пропустить конкретные сроки в файле (в примере ниже пропускаются первая, третья и четвертая строки файла)

In [15]:
pd.read_csv('data/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Обработка отсутствующих значений – важная и зачастую сопровождаемая тонкими нюансами часть разбора файла. Отсутствующие значения обычно либо вообще опущены (пустые строки), либо представлены специальными маркерами. По умолчанию в pandas используется набор общеупотребительных маркеров: NA, -1.#IND и NULL:

In [16]:
result = pd.read_csv('data/ex5.csv')
result

,something,a,b,c,d,message
0,one,NaN,2.0,3.0,4.0,hello
1,two,5.0,NaN,NaN,8.0,world
2,three,9.0,10.0,11.0,NaN,foo


Функция isnull выводит значения True - если ячейка пустая и False - если заполнена

In [17]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,True,False,False,False,False
1,False,False,True,True,False,False
2,False,False,False,False,True,False


Параметр na_values может принимать список или множество строк, рассматриваемых как маркеры отсутствующих значений:

In [18]:
result = pd.read_csv('data/ex5.csv', na_values=['hello', 'world', 'foo'])
result

,something,a,b,c,d,message
0,one,NaN,2.0,3.0,4.0,NaN
1,two,5.0,NaN,NaN,8.0,NaN
2,three,9.0,10.0,11.0,NaN,NaN


Если в разных столбцах применяются разные маркеры, то их можно задать с помощью словаря:

In [19]:
sentinels = {'message': ['foo', 'hello'], 'something': ['two']}
pd.read_csv('data/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,NaN,2.0,3.0,4.0,NaN
1,NaN,5.0,NaN,NaN,8.0,world
2,three,9.0,10.0,11.0,NaN,NaN


В таблице ниже перечислены некоторые часто используемые аргументы функций pandas.read_csv и pandas.read_table.

Чтение текстовых файлов порциями
Для обработки очень больших файлов или для того, чтобы определить правильный набор аргументов, необходимых для обработки большого файла, иногда требуется прочитать небольшой фрагмент файла или последовательно читать файл небольшими порциями.
Прежде чем приступать к обработке большого файла, попросим pandas отображать меньше данных:

In [20]:
pd.options.display.max_rows = 10
result = pd.read_csv('data/ex6.csv', sep='\s+')
result

,one,two,three,four,key
0,0.467976,–0.038649,–0.295344,–1.824726,L
1,–0.358893,1.404453,0.704965,–0.200638,B
2,–0.501840,0.659254,–0.421691,–0.057688,G
3,0.204886,1.074134,1.388361,–0.982404,R
4,0.354628,–0.133116,0.283763,–0.837063,Q
...,...,...,...,...,...
129,2.311896,–0.417070,–1.409599,–0.515821,L
130,–0.479893,–0.650419,0.745152,–0.646038,E
131,0.523331,0.787112,0.486066,1.093156,K
132,–0.362559,0.598894,–1.843201,0.887292,G


Чтобы прочитать только небольшое число строк (а не весь файл), нужно задать это число в параметре nrows:

In [21]:
pd.read_csv('data/ex6.csv', nrows=5, sep='\s+')

,one,two,three,four,key
0,0.467976,–0.038649,–0.295344,–1.824726,L
1,–0.358893,1.404453,0.704965,–0.200638,B
2,–0.501840,0.659254,–0.421691,–0.057688,G
3,0.204886,1.074134,1.388361,–0.982404,R
4,0.354628,–0.133116,0.283763,–0.837063,Q


С помощью метода to_csv объекта DataFrame мы можем вывести данные в файл через запятую:

In [22]:
data = pd.read_csv('data/ex5.csv')
data.to_csv('data/out.csv')

Конечно, допустимы и другие разделители (при выводе в sys.stdout результат отправляется на стандартный вывод, обычно на экран):

In [23]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one||2.0|3.0|4.0|hello
1|two|5.0|||8.0|world
2|three|9.0|10.0|11.0||foo


Отсутствующие значения представлены пустыми строками, но можно вместо этого указать какой-нибудь маркер:

In [24]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,NULL,2.0,3.0,4.0,hello
1,two,5.0,NULL,NULL,8.0,world
2,three,9.0,10.0,11.0,NULL,foo


Если не указано противное, выводятся метки строк и столбцов. Но и те, и другие можно подавить:

In [25]:
data.to_csv(sys.stdout, index=False, header=False)

one,,2.0,3.0,4.0,hello
two,5.0,,,8.0,world
three,9.0,10.0,11.0,,foo


Можно также вывести лишь подмножество столбцов, задав их порядок:

In [26]:
t = data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
,2.0,3.0
5.0,,
9.0,10.0,11.0


У объекта Series также имеется метод to_csv:

In [27]:
import numpy as np
from pandas import Series

dates = pd.date_range('1/1/2000', periods=7)
print(dates)
ts = Series(np.arange(7), index=dates)
print(ts)
ts.to_csv('data/tseries.csv')

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')
2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32


Обработка данных в формате с разделителями
Как правило, табличные данные можно загрузить с диска с помощью функции pandas.read_table и родственных ей. Но иногда требуется ручная обработка. Не так уж необычно встретить файл, в котором одна или несколько строк сформированы неправильно, что сбивает read_table. Для иллюстрации базовых средств рассмотрим небольшой CSV-файл:

"a","b","c"
"1","2","3"
"1","2","3","4"

Для любого файла с односимвольным разделителем можно воспользоваться стандартным модулем Python csv. Для этого передайте открытый файл или объект, похожий на файл, методу csv.reader:

In [28]:
import csv
f = open('data/ex7.csv')
reader = csv.reader(f)
print(reader)

Как видно, переменная reader - это объект.
Итерирование файла с помощью объекта reader дает кортежи значений
в каждой строке после удаления кавычек:

In [29]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


Далее можно произвести любые манипуляции, необходимые для преобразования данных к нужному виду. Пойдем по шагам. Сначала прочитаем файл в список строк:

In [30]:
with open('data/ex7.csv') as f:
    lines = list(csv.reader(f))
print(lines)

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3', '4']]


Затем отделим строку-заголовок от остальных:

In [31]:
header, values = lines[0], lines[1:]

Далее мы можем создать словарь столбцов, применив словарное включение и выражение zip(*values), которое транспонирует строки и столбцы:

In [32]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Для записи файлов с разделителями вручную можно использовать метод csv.writer. Он принимает объект, который представляет открытый, допускающий запись файл и те же параметры диалекта и форматирования, что csv.reader:

In [33]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

with open('data/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))




В библиотеке pandas имеется ряд функций для чтения табличных данных,
представленных в виде объекта DataFrame. Импортируем ее.

In [2]:
import pandas as pd

Прочтем файл ex1.csv в папке data с помощью функции read_csv:

In [21]:
pd.read_csv('data/ex1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Можно было бы также воспользоваться функцией read_table, указав раз-
делитель:

In [20]:
pd.read_table('data/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


В файле не всегда есть строка-заголовок. Рассмотрим файл ex2.csv, содержащий:

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Прочитать его можно двумя способами. Можно поручить pandas выбрать
имена столбцов по умолчанию, а можно задать их самостоятельно:

In [18]:
pd.read_csv('data/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [22]:
pd.read_csv('data/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Допустим, мы хотим, чтобы столбец message стал индексом возвращаемого объекта DataFrame. Этого можно добиться, задав аргумент index_col, в котором указать, что индексом будет столбец с номером 4 или с именем 'message':

In [25]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Если вы хотите сформировать иерархический индекс из нескольких столбцов, то просто передайте список их номеров или имен. Так, например, для файла data/ex3.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16

In [33]:
pd.read_csv('data/ex3.csv', index_col=['key1', 'key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
two  c          5       6
     d          7       8
one  a          9      10
two  b         11      12
one  c         13      14
two  d         15      16

Иногда в таблице нет фиксированного разделителя, а для разделения полей используются пробелы или еще какой-то символ. В таком случае можно передать функции read_table регулярное выражение вместо разделителя. Рассмотрим текстовый файл data/ex3.txt:

         A         B         C
aaa –0.264438 –1.026059 –0.619500
bbb 0.927272 0.302904 –0.032399
ccc –0.264273 –0.386314 –0.217601
ddd –0.871858 –0.348382 1.100491

В данном случае поля разделены переменным числом пробелов и, хотя можно было бы переформатировать данные вручную, проще передать функции read_table регулярное выражение \s+ в качестве разделителя:

In [35]:
pd.read_table('data/ex3.txt', sep='\s+')

,A,B,C
aaa,–0.264438,–1.026059,–0.619500
bbb,0.927272,0.302904,–0.032399
ccc,–0.264273,–0.386314,–0.217601
ddd,–0.871858,–0.348382,1.100491


Поскольку имен столбцов на одно меньше, чем строк, read_table делает вывод, что в данном частном случае первый столбец должен быть индексом DataFrame.

У функций разбора много дополнительных аргументов, которые помогают справиться с широким разнообразием файловых форматов (см. табл.). Так, например, в файле data/ex4.csv, который имеет следующий формат:

# привет!
a,b,c,d,message
# хотелось немного усложнить тебе жизнь,
# а нечего читать CSV–файлы на компьютере
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Параметр skiprows позволяет пропустить конкретные сроки в файле (в примере ниже пропускаются первая, третья и четвертая строки файла)

In [37]:
pd.read_csv('data/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Обработка отсутствующих значений – важная и зачастую сопровождаемая тонкими нюансами часть разбора файла. Отсутствующие значения обычно либо вообще опущены (пустые строки), либо представлены специальными маркерами. По умолчанию в pandas используется набор общеупотребительных маркеров: NA, -1.#IND и NULL:

In [44]:
result = pd.read_csv('data/ex5.csv')
result

,something,a,b,c,d,message
0,one,NaN,2.0,3.0,4.0,hello
1,two,5.0,NaN,NaN,8.0,world
2,three,9.0,10.0,11.0,NaN,foo


Функция isnull выводит значения True - если ячейка пустая и False - если заполнена

In [45]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,True,False,False,False,False
1,False,False,True,True,False,False
2,False,False,False,False,True,False


Параметр na_values может принимать список или множество строк, рассматриваемых как маркеры отсутствующих значений:

In [49]:
result = pd.read_csv('data/ex5.csv', na_values=['hello', 'world', 'foo'])
result

,something,a,b,c,d,message
0,one,NaN,2.0,3.0,4.0,NaN
1,two,5.0,NaN,NaN,8.0,NaN
2,three,9.0,10.0,11.0,NaN,NaN


Если в разных столбцах применяются разные маркеры, то их можно задать с помощью словаря:

In [4]:
sentinels = {'message': ['foo', 'hello'], 'something': ['two']}
pd.read_csv('data/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,NaN,2.0,3.0,4.0,NaN
1,NaN,5.0,NaN,NaN,8.0,world
2,three,9.0,10.0,11.0,NaN,NaN


В таблице ниже перечислены некоторые часто используемые аргументы функций pandas.read_csv и pandas.read_table.

Чтение текстовых файлов порциями
Для обработки очень больших файлов или для того, чтобы определить правильный набор аргументов, необходимых для обработки большого файла, иногда требуется прочитать небольшой фрагмент файла или последовательно читать файл небольшими порциями.
Прежде чем приступать к обработке большого файла, попросим pandas отображать меньше данных:

In [7]:
pd.options.display.max_rows = 10
result = pd.read_csv('data/ex6.csv', sep='\s+')
result

,one,two,three,four,key
0,0.467976,–0.038649,–0.295344,–1.824726,L
1,–0.358893,1.404453,0.704965,–0.200638,B
2,–0.501840,0.659254,–0.421691,–0.057688,G
3,0.204886,1.074134,1.388361,–0.982404,R
4,0.354628,–0.133116,0.283763,–0.837063,Q
...,...,...,...,...,...
129,2.311896,–0.417070,–1.409599,–0.515821,L
130,–0.479893,–0.650419,0.745152,–0.646038,E
131,0.523331,0.787112,0.486066,1.093156,K
132,–0.362559,0.598894,–1.843201,0.887292,G


Чтобы прочитать только небольшое число строк (а не весь файл), нужно задать это число в параметре nrows:

In [12]:
pd.read_csv('data/ex6.csv', nrows=5, sep='\s+')

,one,two,three,four,key
0,0.467976,–0.038649,–0.295344,–1.824726,L
1,–0.358893,1.404453,0.704965,–0.200638,B
2,–0.501840,0.659254,–0.421691,–0.057688,G
3,0.204886,1.074134,1.388361,–0.982404,R
4,0.354628,–0.133116,0.283763,–0.837063,Q


С помощью метода to_csv объекта DataFrame мы можем вывести данные в файл через запятую:

In [30]:
data = pd.read_csv('data/ex5.csv')
data.to_csv('data/out.csv')

Конечно, допустимы и другие разделители (при выводе в sys.stdout результат отправляется на стандартный вывод, обычно на экран):

In [28]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one||2.0|3.0|4.0|hello
1|two|5.0|||8.0|world
2|three|9.0|10.0|11.0||foo


Отсутствующие значения представлены пустыми строками, но можно вместо этого указать какой-нибудь маркер:

In [29]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,NULL,2.0,3.0,4.0,hello
1,two,5.0,NULL,NULL,8.0,world
2,three,9.0,10.0,11.0,NULL,foo


Если не указано противное, выводятся метки строк и столбцов. Но и те, и другие можно подавить:

In [31]:
data.to_csv(sys.stdout, index=False, header=False)

one,,2.0,3.0,4.0,hello
two,5.0,,,8.0,world
three,9.0,10.0,11.0,,foo


Можно также вывести лишь подмножество столбцов, задав их порядок:

In [35]:
t = data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
,2.0,3.0
5.0,,
9.0,10.0,11.0


У объекта Series также имеется метод to_csv:

In [43]:
import numpy as np
from pandas import Series

dates = pd.date_range('1/1/2000', periods=7)
print(dates)
ts = Series(np.arange(7), index=dates)
print(ts)
ts.to_csv('data/tseries.csv')

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')
2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32


Обработка данных в формате с разделителями
Как правило, табличные данные можно загрузить с диска с помощью функции pandas.read_table и родственных ей. Но иногда требуется ручная обработка. Не так уж необычно встретить файл, в котором одна или несколько строк сформированы неправильно, что сбивает read_table. Для иллюстрации базовых средств рассмотрим небольшой CSV-файл:

"a","b","c"
"1","2","3"
"1","2","3","4"

Для любого файла с односимвольным разделителем можно воспользоваться стандартным модулем Python csv. Для этого передайте открытый файл или объект, похожий на файл, методу csv.reader:

In [44]:
import csv
f = open('data/ex7.csv')
reader = csv.reader(f)
print(reader)

Как видно, переменная reader - это объект.
Итерирование файла с помощью объекта reader дает кортежи значений
в каждой строке после удаления кавычек:

In [45]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


Далее можно произвести любые манипуляции, необходимые для преобразования данных к нужному виду. Пойдем по шагам. Сначала прочитаем файл в список строк:

In [52]:
with open('data/ex7.csv') as f:
    lines = list(csv.reader(f))
print(lines)

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3', '4']]


Затем отделим строку-заголовок от остальных:

In [53]:
header, values = lines[0], lines[1:]

Далее мы можем создать словарь столбцов, применив словарное включение и выражение zip(*values), которое транспонирует строки и столбцы:

In [55]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Для записи файлов с разделителями вручную можно использовать метод csv.writer. Он принимает объект, который представляет открытый, допускающий запись файл и те же параметры диалекта и форматирования, что csv.reader:

In [59]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

with open('data/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))


